# <span style="color:blue; font-weight:bold">FINAL PROJECT</span>

# 1.Importation

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, PolynomialFeatures
# from sklearn.compose import ColumnTransformer
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import r2_score

import warnings

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Dataset properties (owner/dataset-name)
dataset_name = "davidgauthier/glassdoor-job-reviews"

# Download the dataset files to the current directory
api.dataset_download_files(dataset_name, path='./', unzip=True)

In [ ]:
# Read CSV file into a DataFrame
dataset = pd.read_csv("glassdoor_reviews.csv")
dataset_full = dataset.copy()

In [ ]:
# Display the first few rows of the dataset to understand its structure
print(dataset.head(3))

Ranking for the recommendation of the firm, CEO approval, and outlook are allocated categories v, r, x, and o, with the following meanings:
v - Positive, r - Mild, x - Negative, o - No opinion

# 2. EDA

In [ ]:
dataset.info()

In [ ]:
total_length = len(dataset)
print(f"Total length of the dataset: {total_length}")

## 2.1 Valeurs manquantes

In [ ]:
# Check for missing values in the dataset
missing_values = dataset.isnull().sum()
print("Nombres de valeurs manquantes:\n", total_length - missing_values[missing_values > 0])
print()
percentage = (missing_values / total_length) * 100
print(f"Pourcentage de valeurs manquantes:\n{percentage[missing_values > 0].sort_values(ascending=False)}")

Avec 83% de données manquantes, on supprime diversity_inclusion

In [ ]:
# Drop de diversity_inclusion
dataset.drop(columns=["diversity_inclusion"], inplace=True)

In [ ]:
list_numerical = dataset.select_dtypes(include=[np.number]).columns.tolist()
list_categorical = dataset.select_dtypes(include=[np.object_]).columns.tolist()

## 2.2 Firmes' reviews

In [ ]:
df_firm = dataset["firm"].reset_index(drop=True)
firm_count = df_firm.value_counts()
# Nombre de firmes
print(f"Nombre de firmes: {len(firm_count)}")
# Nombre de firmes avec au moins 10 reviews
review = 100
print(f"Nombre de firmes avec au moins {review} reviews: {len(firm_count[firm_count >= review])}")


In [ ]:
# Top X firmes avec le plus de reviews
top_nb = 30
top_firmes = firm_count.nlargest(top_nb)
top_firmes_normalised = 100 * top_firmes/top_firmes.max()

# J'ai normalisé les reviews pour comparer les firmes entre elles
fig = px.histogram(top_firmes, x=top_firmes.index, y=top_firmes.values)
fig.update_layout(
    title=f"Top {top_nb} firmes avec le plus de reviews",
    xaxis_title="Firme",
    yaxis_title="Nombres de reviews",
    xaxis_tickangle=-45,
)
fig.show()

fig = px.histogram(top_firmes_normalised, x=top_firmes_normalised.index, y=top_firmes_normalised.values)
fig.update_layout(
    title=f"Top {top_nb} firmes avec le plus de reviews",
    xaxis_title="Firme",
    yaxis_title="Pourcentages de reviews (%) normalisés",
    xaxis_tickangle=-45,
)
fig.show()

Je propose de limiter le projet aux 30 plus grandes entreprises

In [ ]:
# dataset_limited = dataset[dataset["firm"].isin(top_firmes.index.to_list())].reset_index(drop=True)
# dataset = dataset_limited

Gestion des erreurs manquantes pour les variables numériques

In [ ]:
dataset_drop = dataset.copy()
dataset_replacement = dataset.copy()

In [ ]:
# Drop des erreurs manquantes
for feature in list_numerical:
    dataset_drop = dataset_drop.dropna(subset=feature, how="all")
dataset_drop.reset_index(drop=True, inplace=True)

In [ ]:
# Remplacement des valeurs manquantes par la valeur médiane pour chaque entreprise
for feature in list_numerical:
    dataset_replacement[feature] = dataset_replacement.groupby("firm")[feature].transform(lambda df: df.fillna(df.median()))

In [ ]:
df_firm_drop = dataset_drop["firm"].reset_index(drop=True).value_counts()
df_firm_drop = df_firm_drop.nlargest(top_nb).reset_index()

df_firm_replacement = dataset_replacement["firm"].reset_index(drop=True).value_counts()
df_firm_replacement = df_firm_replacement.nlargest(top_nb).reset_index()


In [ ]:
df = pd.merge(df_firm_drop,df_firm_replacement,on='firm')
df["diff Y-X"] = round(100 - df["count_x"]*100/df["count_y"],1)
df

On observe des pertes conséquentes si on drop les données manquantes.

In [ ]:
dataset = dataset_replacement.copy()

In [ ]:
dataset["current"].unique()

On va créer de nouvelles catégories: Current & Former; Employee, Contractor, Intern, Freelancer, Temporary

In [ ]:
def categorize_experience(string_value):
    if 'less than 1 year' in string_value.lower():
        return '1-'
    elif 'more than 1 year' in string_value.lower():
        return '1+'
    elif 'more than 3 year' in string_value.lower():
        return '3+'
    elif 'more than 5 year' in string_value.lower():
        return '5+'
    elif 'more than 8 year' in string_value.lower():
        return '8+'
    elif 'more than 10 year' in string_value.lower():
        return '10+'
    else:
        return 'No information'

In [ ]:
def categorize_job(string_value):
    if 'employee' in string_value.lower():
        return 'employee'
    elif 'contractor' in string_value.lower():
        return 'contractor'
    elif 'intern' in string_value.lower():
        return 'intern'
    elif 'temporary' in string_value.lower():
        return 'temporary'
    elif 'freelancer' in string_value.lower():
        return 'freelancer'
    else:
        return np.nan

In [ ]:
def categorize_current(string_value):
    if 'current' in string_value.lower():
        return 'current'
    elif 'former' in string_value.lower():
        return 'former'
    else:
        return np.nan

In [ ]:
dataset["status"] = dataset["current"].apply(categorize_current)
dataset["job"] = dataset["current"].apply(categorize_job)
dataset["experience"] = dataset["current"].apply(categorize_experience)


In [ ]:
fig = px.pie(dataset, names='experience', title='Répartition des experiences')
fig.show()

In [ ]:
fig = px.histogram(dataset, x = dataset['job'], title='Répartition des jobs')
fig.show()

Suppresion des cases autres que employee ?

In [ ]:
fig = px.pie(dataset, names = dataset['status'], title='Répartition des status')
fig.show()

In [ ]:
dataset.head(1)

In [ ]:
dataset.drop(columns=["current"], inplace=True)

In [ ]:
list_review = ['headline','pros','cons']
dataset_review = dataset[list_review].copy()
dataset_notes = dataset.drop(columns=list_review).copy()

In [ ]:
len(dataset_notes['job_title'].unique())